In [1]:
from petrify import u, tau
from petrify.formats import STL

This is the original:

In [2]:
original = STL.read('original-dial-mount.stl', 'mm').m_as('mm')
original.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

The mounting screws are in the right place for the carriage, but the screws for the dial indicator are not remotely compatible with my model.

This is easily fixed though. Other parts that work with my dial indicator just hold the center bore:

<img src="./other-holder.jpg" width=600px />

So, I cut away the top part of the mount to give some clearance for the indicator:

In [3]:
from petrify.solid import Point, Vector, Collection, Box, Cylinder

cut = Box(Point(-30, -20, 10), Vector(20, 20, 50))

Collection([original, cut.view(color='#f00')]).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

In [4]:
ready_for_bore = (original - cut)
ready_for_bore.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

I measured the above holder. It's clearly tapered, with a top diameter of 8.4mm and a bottom diameter of 7.9mm. So I created a tapered cylinder using the [`Spun`](https://petrify.readthedocs.io/en/latest/petrify.solid.html#petrify.solid.Spun) solid:

In [5]:
from petrify import plane
from petrify.solid import Spun

origin = Point(-20, -8, 0)

br = 7.9 / 2
tr = 8.4 / 2
height = 15
axis = Vector.basis.z
start = Vector.basis.y
taper = plane.Polygon([
    plane.Point(0, 0), 
    plane.Point(br, 0),
    plane.Point(tr, height),
    plane.Point(0, height)
])
bore = Spun(axis, start, [taper] * 30) + origin

Collection([ready_for_bore, bore.view(color='#f00')]).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

This clearly won't work though, the extension arm isn't wide enough. So I added an additional support cylinder to wrap around the bore:

In [6]:
support = Cylinder(origin, Vector.basis.z * height, 12/2, segments=30)
bored = (ready_for_bore + support) - bore
bored.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

Finally, the original holder had a small channel removed from the holder to allow the plastic to flex if a tighter fit is desired. So I positioned that:

In [7]:
channel = Box(origin - Vector(1, 0, 0), Vector(2, 10, 20)).rotate_at(origin, Vector.basis.z, tau / 2)
Collection([bored, channel.view(color='#f00')]).render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

And then cut it away:

In [8]:
final = bored - channel
final.render()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

Now, it's ready for export, slicing, and printing:

In [9]:
STL('../output/mount.stl', u.mm).write(final.as_unit(u.mm))